## Modeling

In [ ]:
%set_env TF_USE_LEGACY_KERAS=True

In [ ]:
import pandas as pd
import tf_keras as keras 

In [ ]:
reviews = pd.read_parquet("../data/2023/Electronics_min.parquet")
items = pd.read_parquet("../data/2023/meta_Electronics.parquet")

In [ ]:
reviews

In [ ]:
items

We need to build a compact user-centric representation of preference, collapse review data into a sparse matrix of user -> item preferences. There are some heuristics that need to be applied in the process: 
1. users with few interactions are a very weak signal -- without associations with multiple products, we are not teaching the model about positive associations
2. products with few interactions are also a very weak signal -- we are looking to connect users and items that have tiny interaction graphs are not going to improve our macro-level predictions

In [ ]:
# TODO: This should be a configuration option hyper parameter, we can relax this if training isn't suepr computationally expensive
min_ratings = 1000
items = items[items.rating_number > min_ratings]
len(items) 

In [ ]:
# The above item filtering serves to reduce the computational complexity as well as 
# reduce sparsity, before we filter reviews make sure we remove those associated with 
# dropped items
all_items = set(items.parent_asin)
reviews = reviews[reviews.parent_asin.isin(all_items)]
reviews.rename(columns={'parent_asin':'item_id'}, inplace=True)

In [ ]:
users = reviews.groupby(['user_id']).rating.count()
users = pd.DataFrame(users).reset_index()
users.rename(columns={'rating':'ratings'}, inplace=True)

In [ ]:
# This is a configuration parameter, as above ... relax if we don't have issues with compute
min_reviews = 10
users = users[users.ratings > min_reviews] 
users

❗in the notebook, ratings are thresholded ... do we need to follow suit? what are the ramifications if we don't? OH... in the notebook, a click is an interaction, there's no middle ground. the network is going to operate on 0s or 1s. by leaving low reviews in our matrix, the network would learn to recommend things users have interacted with, but not necessarily positively. our case is the same, a review is an interaction. we're aiming to recommend, and we should not want to recommend low reviews. so filter... 

In [ ]:
#TODO: decide if we need to keep the low reviews around 
reviews = reviews[reviews.rating >= 3]

In [ ]:
# Discard reviews by users outside our core group 
reviews = reviews[reviews.user_id.isin(set(users.user_id))]

In [ ]:
reviews

In [ ]:
matrix_size = len(users) * len(items)
matrix_size

In [ ]:
from recommenders.datasets.sparse import AffinityMatrix

In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
reviews.columns

In [ ]:
reviews

In [ ]:
# NOTE: Strategy adapted from tutorials available in the Recommenders project, see 
# https://github.com/recommenders-team/recommenders/tree/main
from recommenders.datasets.python_splitters import python_random_split

# Split along user boundaries to ensure no leakage of preference between train and test
train_users, test_users, val_users = python_random_split(users, [.9, .05, .05])

In [ ]:
print(train_users.shape, test_users.shape, val_users.shape)

In [ ]:
train = reviews[reviews.user_id.isin(train_users.user_id)]
val = reviews[reviews.user_id.isin(val_users.user_id)]
test = reviews[reviews.user_id.isin(test_users.user_id)]

In [ ]:
print(train.shape, val.shape, test.shape)

In [ ]:
# Technique from Recommenders (see https://github.com/recommenders-team/recommenders/blob/45e1b215a35e69b92390e16eb818d4528d0a33a2/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb) 
# to improve utility of validation set during training - only allow items in
# the validation set that are also present in the train set
val = val[val.item_id.isin(train.item_id.unique())]

In [ ]:
val.shape

In [ ]:
from recommenders.datasets.python_splitters import python_stratified_split 

# Another technique employed in Recommenders (see above link for notebook), for in-flight validation to be 
# meaningful during training, our validation set needs not just ground truth, but unseen validation samples 
# to see if predictions for validation users are relevant (to those users). Anyway, break down our val and test 
# sets again to support this strategy
val_src, val_target = python_stratified_split(
    data=val, 
    ratio=0.8, 
    filter_by="item", 
    col_user="user_id", 
    col_item="item_id"
    )
test_src, test_target = python_stratified_split(
    data=test, 
    ratio=0.8, 
    filter_by="item", 
    col_user="user_id", 
    col_item="item_id"
    )

In [ ]:
print(val.shape, " -> ", val_src.shape, val_target.shape)
print(test.shape, " -> ", test_src.shape, test_target.shape)

In [ ]:
#to use standard names across the analysis 
header = {
        "col_user": "user_id",
        "col_item": "item_id",
        "col_rating": "rating",
        # Unclear why this doesn't also eat a timestamp, but many of the functions that split temporally use, fortunately 
        # the column 'timestamp' (i.e. DEFAULT_TIMESTAMP_COL='timestamp') so I think we're fine. 
        # "col_timestamp" : "timestamp"
    }

train_matrix = AffinityMatrix(df=train, **header)
val_src_matrix = AffinityMatrix(df=val_src, **header)
val_tgt_matrix = AffinityMatrix(df=val_target, **header)
test_src_matrix = AffinityMatrix(df=test_src, **header)
test_tgt_matrix = AffinityMatrix(df=test_target, **header)

In [ ]:
# This generates a sparse array of user vectors, aka user-item matrix
# X[0] is the first user in the list, with entries for all items known when the matrix was constructed in that row
train, _, _ = train_matrix.gen_affinity_matrix()
val_src, _, _ = val_src_matrix.gen_affinity_matrix()
val_tgt, _, _ = val_tgt_matrix.gen_affinity_matrix()
test_src, _, _ = test_src_matrix.gen_affinity_matrix()
test_tgt, _, _ = test_src_matrix.gen_affinity_matrix()

In [ ]:
from recommenders.utils.python_utils import binarize

train = binarize(train, 3)
val_src = binarize(val_src, 3) 
val_tgt = binarize(val_tgt, 3)
test_src = binarize(test_src, 3)
test_tgt = binarize(test_tgt, 3)

In [ ]:
# TODO: Make sure this is reported during training/configuration
sparsity = np.count_nonzero(train)/(train.shape[0]*train.shape[1])
sparsity

## Model Design 

In [ ]:
keras.__version__ 

In [ ]:
from recommenders.models.vae.standard_vae import StandardVAE

In [ ]:
model = StandardVAE(
    n_users = train.shape[0], 
    original_dim = train.shape[1],
    intermediate_dim=250, 
    latent_dim=50, 
    n_epochs=1, 
    batch_size=1, 
    k=10, 
    verbose=1, 
    seed=4, 
    save_path="models/svae.hdf5", 
    drop_encoder=0.5, 
    drop_decoder=0.5, 
    annealing=False, 
    beta=1.0) 

In [ ]:
model.fit(x_train=train, 
          x_valid=val, 
          x_val_tr=val_src, 
          x_val_te=val_tgt, 
          mapper=AffinityMatrix(val),
          )

Managing text-based reviews at this scale could be a challenge, and I'd like to steer clear of LLMs for this effort. We could do an embedding on the review and use that for similarity, but we have pretty rich item data. Perhaps let's ignore the collaborative aspect here and build a shopping interface that: 
- surfaces the most popular items, and encourages you to add items to your shopping cart for a big discount/promo
- based on clicks and cart items, improves the recommendations and surfaces new products

We can use an autoencoder to accept a sparse matrix of users and items, learn to reproduce that matrix, and in so doing support prediction on missing values. However, this matrix is of size users x items, which here is 1.8e7 x 1.6e6 = 28,125,000,000 KB (best-case, higher if stored as np floats) ~= 26 TB !! WTF. 
- In the standard VAE example (https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb) the clicks are turned into a histogram for each user ... so we have n_user vector of length n_items... then I guess each of these is a training sample. The VAE presumably learns, given a sparse user vector, to predict every rating. This takes the complexity down and gives us a training set we can iterate over. 

Let's avoid any distributional pressure (present in VAE, SVAE, disentangled VAE) and go for a basic autoencoder using the strategy laid out above, i.e.

## Autoencoder Prototype

In [ ]:
import torch 
import pandas as pd 
import torch
from tqdm import tqdm
import os
import math 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb

In [ ]:
class Autoencoder(nn.Module):
    """
    Autoencoder

    NOTE: with cues from https://www.geeksforgeeks.org/deep-learning/implementing-an-autoencoder-in-pytorch/
    """

    def __init__(self, dims=1000):
        """
        Initialize a new object 
        """
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(dims, 500),
            nn.Linear(500, 75),
        )
        self.decoder = nn.Sequential(
            nn.Linear(75, 500),
            nn.Linear(500, dims),
        )

    def forward(self, x):
        """
        Implement our forward pass 
        """
        h = self.encoder(x) 
        r = self.decoder(h)

        return r

In [ ]:
class DeepCartDataset(torch.utils.data.Dataset): 
    """
    Custom pytorch-compatible dataset. Adapted from 
    https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files
    """
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None): 

        self.img_labels = pd.read_csv(annotations_file)

        #TODO: implement

    def __len__(self): 
        return len(self.img_labels) 
    
    def __getitem__(self, idx): 
        #TODO: implement
        pass

In [ ]:
def get_data_loader(batch_size=5, shuffle=True): 
    """
    Retrieve a pytorch-style dataloader 
    """

    #TODO: implement
    #transform = transforms.Compose([
    #     transforms.ConvertImageDtype(torch.float),
    #     transforms.Normalize(mean=[0.5], std=[0.5])
    #])

    #data = DeepCartDataset(transform=transform)
    #loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=shuffle)
    
    #return loader
    pass

In [ ]:
def train(loader, model, loss_interval=20, epochs=2, lr=0.01, momentum=0.9):
    """
    Train the model with the provided dataset

    NOTE: this is a similar training loop as we used for our vision model in the 
    the vision project, forward pass
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    train_loss = []

    tqdm.write(f"Starting training run...")    
    # TODO: configure WandB
    # see https://docs.wandb.ai/guides/integrations/pytorch/
    config = {}
    run = wandb.init(config=config) 

    model.train()
    model = model.to(device)
    
    loss_fn = nn.CrossEntropyLoss()

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(epochs):

        running_loss = 0.0
        for i, data in enumerate(loader):

            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            # collect metrics
            running_loss += loss.item()

            if (i % loss_interval) == (loss_interval - 1): 
                train_loss.append(running_loss / loss_interval)
                tqdm.write(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / loss_interval:.3f}")
                running_loss = 0 
    
    tqdm.write("Training complete!") 

    return train_loss 